<H1>Problem Statement</H1>
Your client is a large MNC and they have 9 broad verticals across the organisation. One of the problem your client is facing is around identifying the right people for promotion (only for manager position and below) and prepare them in time. Currently the process, they are following is:

They first identify a set of employees based on recommendations/ past performance
Selected employees go through the separate training and evaluation program for each vertical. These programs are based on the required skill of each vertical
At the end of the program, based on various factors such as training performance, KPI completion (only employees with KPIs completed greater than 60% are considered) etc., employee gets promotion
For above mentioned process, the final promotions are only announced after the evaluation and this leads to delay in transition to their new roles. Hence, company needs your help in identifying the eligible candidates at a particular checkpoint so that they can expedite the entire promotion cycle.

They have provided multiple attributes around Employee's past and current performance along with demographics. Now, The task is to predict whether a potential promotee at checkpoint in the test set will be promoted or not after the evaluation process.

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!cp  /content/drive/Hackthon/WNS/* .
!ls -l

In [1]:
!ls -l


total 5556
-rw-r--r-- 1 root root    2495 Sep 15 07:51 adc.json
drwxr-xr-x 4 root root    4096 Sep 15 08:08 catboost_info
drwxr-xr-x 2 root root    4096 Sep 15 07:51 drive
drwxr-xr-x 2 root root    4096 Sep 13 17:28 sample_data
-rw-r--r-- 1 root root 1565147 Sep 15 08:57 test_2umaH9m.csv
-rw-r--r-- 1 root root 3759647 Sep 15 08:57 train_LZdllcl.csv
-rw-r--r-- 1 root root     229 Sep 15 08:57 Untitled
-rw-r--r-- 1 root root  211100 Sep 15 08:57 WNS_Traine_01.ipynb
-rw-r--r-- 1 root root   62371 Sep 15 08:57 WNS_Traine_02.ipynb
-rw-r--r-- 1 root root   57610 Sep 15 08:57 WNS_Traine_03.ipynb


In [0]:
!pip install catboost
!pip install ipywidgets
!pip install lightgbm
!pip install xgboost     


<h1>3. Exploratory Data Analysis</h1>
<h3>3.1. Import All required Classed first</h3>


In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt


from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix,roc_auc_score,mean_squared_error,f1_score,recall_score,precision_score
from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.cross_validation import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from numpy import argmax
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool, cv
from lightgbm import LGBMClassifier

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [0]:
SEED = 0

In [0]:
Data = pd.read_csv("train_LZdllcl.csv")

In [5]:
print(Data.shape)
print(Data.columns)

(54808, 14)
Index(['employee_id', 'department', 'region', 'education', 'gender',
       'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating',
       'length_of_service', 'KPIs_met >80%', 'awards_won?',
       'avg_training_score', 'is_promoted'],
      dtype='object')


In [6]:
print(Data.describe())

        employee_id  no_of_trainings           age  previous_year_rating  \
count  54808.000000     54808.000000  54808.000000          50684.000000   
mean   39195.830627         1.253011     34.803915              3.329256   
std    22586.581449         0.609264      7.660169              1.259993   
min        1.000000         1.000000     20.000000              1.000000   
25%    19669.750000         1.000000     29.000000              3.000000   
50%    39225.500000         1.000000     33.000000              3.000000   
75%    58730.500000         1.000000     39.000000              4.000000   
max    78298.000000        10.000000     60.000000              5.000000   

       length_of_service  KPIs_met >80%   awards_won?  avg_training_score  \
count       54808.000000   54808.000000  54808.000000        54808.000000   
mean            5.865512       0.351974      0.023172           63.386750   
std             4.265094       0.477590      0.150450           13.371559   
min    

In [7]:
Data.count()

employee_id             54808
department              54808
region                  54808
education               52399
gender                  54808
recruitment_channel     54808
no_of_trainings         54808
age                     54808
previous_year_rating    50684
length_of_service       54808
KPIs_met >80%           54808
awards_won?             54808
avg_training_score      54808
is_promoted             54808
dtype: int64

In [0]:
values={"previous_year_rating":Data.previous_year_rating.mean(),"education":"O"}
Data.fillna(value=values,inplace=True)

In [9]:
Data.isna().values.any()


False

In [10]:
print(len(Data))
print(len(Data.loc[Data["is_promoted"]==0]))
print(len(Data.loc[Data["is_promoted"]==1]))
print(len(Data))

54808
50140
4668
54808


In [11]:
50140/54808*100

91.48299518318494

In [12]:
Data.count()

employee_id             54808
department              54808
region                  54808
education               54808
gender                  54808
recruitment_channel     54808
no_of_trainings         54808
age                     54808
previous_year_rating    54808
length_of_service       54808
KPIs_met >80%           54808
awards_won?             54808
avg_training_score      54808
is_promoted             54808
dtype: int64

In [13]:
Data.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [0]:
data_xs = Data.drop(['employee_id',"is_promoted"],axis=1)


In [16]:
data_xs.head()


,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49
1,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60
2,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50
3,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50
4,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73


In [0]:
categorical_features_indices = np.array([0,1,2,3,4,6,8,10,11])
#categorical_features_indices = np.array([0,1,3,6,7])

In [0]:
def model_evalution(model):
  print("####################### model Evalution started #######################")
  train_pre = model.predict(x_train)
  test_pre = model.predict(x_test)
  train_pro = model.predict_proba(x_train)
  test_pro = model.predict_proba(x_test)
  
  print("Train Accuracy: {0} \t Test Accuracy: {1}".format(accuracy_score(y_train, train_pre),accuracy_score(y_test,test_pre)))
  print("Train Loss: {0} \t Test Loss: {1}".format(mean_squared_error(y_train, train_pre),mean_squared_error(y_test,test_pre)))
  print("Train AUC: {0} \t Test AUC: {1}".format(roc_auc_score(y_train, train_pro[:,1]),roc_auc_score(y_test,test_pro[:,1])))
  print("Train F1: {0} \t Test F1: {1}".format(f1_score(y_train, train_pre),f1_score(y_test,test_pre)))
  print("Train recall: {0} \t Test recall: {1}".format(recall_score(y_train, train_pre),recall_score(y_test,test_pre)))
  print("Train precision: {0} \t Test Precision: {1}".format(precision_score(y_train, train_pre),precision_score(y_test,test_pre)))
  print("Train Confusion Matrix: \n{0} \n Test Confusion Matrix: \n{1}".format(confusion_matrix(y_train, train_pre),confusion_matrix(y_test,test_pre)))
  #f1_score
  
  

In [0]:
x_train,x_test,y_train,y_test = train_test_split(data_xs,Data["is_promoted"], test_size = 0.3,random_state=SEED)

In [0]:
del(catboost_model)

In [0]:
catboost_model = CatBoostClassifier(iterations=100,verbose=True,eval_metric="F1",learning_rate=0.5,class_weights=[1,3],depth=3,l2_leaf_reg=50,bagging_temperature=0.5) #--->0.5139
#catboost_model.fit(x_train, y_train,cat_features=categorical_features_indices,eval_set=(x_test, y_test))
catboost_model.fit(x_train, y_train,cat_features=categorical_features_indices)
model_evalution(catboost_model)

0:	learn: 0.1996516	total: 144ms	remaining: 50.2s
1:	learn: 0.5114135	total: 272ms	remaining: 47.3s
2:	learn: 0.5181046	total: 389ms	remaining: 45s
3:	learn: 0.5159458	total: 506ms	remaining: 43.8s
4:	learn: 0.5149123	total: 628ms	remaining: 43.3s
5:	learn: 0.5157518	total: 743ms	remaining: 42.6s
6:	learn: 0.5162516	total: 865ms	remaining: 42.4s
7:	learn: 0.5193652	total: 987ms	remaining: 42.2s
8:	learn: 0.5229898	total: 1.11s	remaining: 42s
9:	learn: 0.5233672	total: 1.23s	remaining: 42s
10:	learn: 0.5342554	total: 1.35s	remaining: 41.5s
11:	learn: 0.5352715	total: 1.48s	remaining: 41.6s
12:	learn: 0.5352715	total: 1.57s	remaining: 40.8s
13:	learn: 0.5355845	total: 1.67s	remaining: 40s
14:	learn: 0.5355464	total: 1.73s	remaining: 38.7s
15:	learn: 0.5348440	total: 1.85s	remaining: 38.6s
16:	learn: 0.5348821	total: 1.96s	remaining: 38.5s
17:	learn: 0.5348821	total: 2.08s	remaining: 38.4s
18:	learn: 0.5384452	total: 2.2s	remaining: 38.3s
19:	learn: 0.5406593	total: 2.32s	remaining: 38.3s

In [0]:
####################### model Evalution started #######################
Train Accuracy: 0.9342108692818976 	 Test Accuracy: 0.9354132457580734
Train Loss: 0.06578913071810244 	 Test Loss: 0.06458675424192666
Train AUC: 0.9171867009622848 	 Test AUC: 0.91122982306485
Train F1: 0.5275177836016474 	 Test F1: 0.528
Train recall: 0.4293113954905545 	 Test recall: 0.42857142857142855
Train precision: 0.6839805825242719 	 Test Precision: 0.6875
Train Confusion Matrix: 
[[34432   651]
 [ 1873  1409]] 
 Test Confusion Matrix: 
[[14787   270]
 [  792   594]]





Shrink model to first 195 iterations.
####################### model Evalution started #######################
Train Accuracy: 0.9317346539814936 	 Test Accuracy: 0.9316426442863225
Train Loss: 0.06826534601850645 	 Test Loss: 0.06835735571367756
Train AUC: 0.920548885351588 	 Test AUC: 0.9105997977287078
Train F1: 0.535391165513571 	 Test F1: 0.5347682119205298
Train recall: 0.45978062157221206 	 Test recall: 0.4660894660894661
Train precision: 0.640764331210191 	 Test Precision: 0.6271844660194175
Train Confusion Matrix: 
[[34237   846]
 [ 1773  1509]] 
 Test Confusion Matrix: 
[[14673   384]
 [  740   646]]



Shrink model to first 192 iterations.
####################### model Evalution started #######################
Train Accuracy: 0.9299882705591034 	 Test Accuracy: 0.9255610290093049
Train Loss: 0.07001172944089665 	 Test Loss: 0.07443897099069513
Train AUC: 0.924864515163944 	 Test AUC: 0.9096398572389807
Train F1: 0.5535239361702128 	 Test F1: 0.5248447204968943
Train recall: 0.5073126142595978 	 Test recall: 0.48773448773448774
Train precision: 0.6089978054133138 	 Test Precision: 0.5680672268907563
Train Confusion Matrix: 
[[34014  1069]
 [ 1617  1665]] 
 Test Confusion Matrix: 
[[14543   514]
 [  710   676]]

In [0]:
catboost_model.feature_importances_

array([17.59353534,  3.04785304,  1.04918581,  0.23318956,  0.81186317,
        0.59054397,  3.27616503,  6.46528825,  2.201155  , 34.88830721,
        1.37520843, 28.46770519])

In [0]:
cat_features=categorical_features_indices,eval_set=(x_test, y_test)

In [0]:
Data_test = pd.read_csv("test_2umaH9m.csv")

In [0]:
print(Data_test.shape)
print(Data_test.columns)

(23490, 13)
Index(['employee_id', 'department', 'region', 'education', 'gender',
       'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating',
       'length_of_service', 'KPIs_met >80%', 'awards_won?',
       'avg_training_score'],
      dtype='object')


In [0]:
print(Data_test.describe())

        employee_id  no_of_trainings           age  previous_year_rating  \
count  23490.000000     23490.000000  23490.000000          21678.000000   
mean   39041.399149         1.254236     34.782929              3.339146   
std    22640.809201         0.600910      7.679492              1.263294   
min        3.000000         1.000000     20.000000              1.000000   
25%    19370.250000         1.000000     29.000000              3.000000   
50%    38963.500000         1.000000     33.000000              3.000000   
75%    58690.000000         1.000000     39.000000              4.000000   
max    78295.000000         9.000000     60.000000              5.000000   

       length_of_service  KPIs_met >80%   awards_won?  avg_training_score  
count       23490.000000   23490.000000  23490.000000        23490.000000  
mean            5.810387       0.358834      0.022776           63.263133  
std             4.207917       0.479668      0.149191           13.411750  
min        

In [0]:
Data.count()

employee_id             54808
department              54808
region                  54808
education               54808
gender                  54808
recruitment_channel     54808
no_of_trainings         54808
age                     54808
previous_year_rating    54808
length_of_service       54808
KPIs_met >80%           54808
awards_won?             54808
avg_training_score      54808
is_promoted             54808
dtype: int64

In [0]:
values={"previous_year_rating":Data_test.previous_year_rating.mean(),"education":"O"}
Data_test.fillna(value=values,inplace=True)

In [0]:
Data_test.isna().values.any()


False

In [0]:
Data_test.count()

employee_id             23490
department              23490
region                  23490
education               23490
gender                  23490
recruitment_channel     23490
no_of_trainings         23490
age                     23490
previous_year_rating    23490
length_of_service       23490
KPIs_met >80%           23490
awards_won?             23490
avg_training_score      23490
dtype: int64

In [0]:
Data_test.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,3.339146,1,1,0,77
1,74430,HR,region_4,Bachelor's,f,other,1,31,3.000000,5,0,0,51
2,72255,Sales & Marketing,region_13,Bachelor's,m,other,1,31,1.000000,4,0,0,47
3,38562,Procurement,region_2,Bachelor's,f,other,3,31,2.000000,9,0,0,65
4,64486,Finance,region_29,Bachelor's,m,sourcing,1,30,4.000000,7,0,0,61


In [0]:
data_test_xs = Data_test.drop(['employee_id'],axis=1)
#data_test_xs = Data_test.drop(['employee_id',"education","gender","recruitment_channel","region"],axis=1)

In [0]:
y_test_pre = catboost_model.predict(data_test_xs)

In [0]:
Data_test["is_promoted"] = y_test_pre

In [0]:
Data_test.to_csv("WNS_catBoost_Test_05_30.csv",columns=["employee_id","is_promoted"],index=False)

In [0]:
from google.colab import files
files.download("WNS_catBoost_Test_05_30.csv")